In [1]:
import pandas as pd
#LOAD DATA
print("Loading data...")
train = pd.read_table('../../dat/train.tsv')
test = pd.read_table('../../dat/test.tsv')
print('Data loaded')
print(train.shape)
print(test.shape)

Loading data...
Data loaded
(1482535, 8)
(693359, 7)


In [2]:
import numpy as np
train['target'] = np.log(train['price']+1)# same as np.log1p(train_df['price'])


In [3]:
#HANDLE MISSING VALUES
print("Handling missing values...")
def handle_missing(dataset):
    dataset.category_name.fillna(value="missing", inplace=True)
    dataset.brand_name.fillna(value="missing", inplace=True)
    dataset.item_description.fillna(value="missing", inplace=True)
    return (dataset)

train = handle_missing(train)
test = handle_missing(test)
print(train.shape)
print(test.shape)

Handling missing values...
(1482535, 9)
(693359, 7)


In [9]:
from sklearn.cross_validation import train_test_split
#EXTRACT DEVELOPTMENT TEST
dtrain, dvalid = train_test_split(train, random_state=123, train_size=0.99)
print(dtrain.shape)
print(dvalid.shape)

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(1467709, 10)
(14826, 10)


In [13]:
dvalid.head(10)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,target,category_brand
933947,933947,On hold for Chelseychu,2,Women/Women's Handbags/Shoulder Bag,Hello Kitty,31.0,0,Black faux leather hello kitty satchel comes w...,3.465736,Women/Women's Handbags/Shoulder Bag/Hello Kitty
236612,236612,Lace up leggings,2,Women/Pants/Casual Pants,missing,15.0,1,Only worn for picture,2.772589,Women/Pants/Casual Pants/missing
832246,832246,Zipper High Top All Star Converse,3,Women/Shoes/Fashion Sneakers,Converse,26.0,0,Size 7 women's Great for your feet! Check out ...,3.295837,Women/Shoes/Fashion Sneakers/Converse
343557,343557,Sperry,3,Kids/Girls 2T-5T/Shoes,missing,16.0,0,Pink Size 6 toddler Used Very good condition G...,2.833213,Kids/Girls 2T-5T/Shoes/missing
1227445,1227445,Lipsense set,1,Beauty/Makeup/Lips,SeneGence,50.0,1,Bella color with Glossy gloss and oops remover,3.931826,Beauty/Makeup/Lips/SeneGence
787573,787573,Silver tulle,1,Kids/Toys/Arts & Crafts,missing,7.0,0,"Two 6"" x 10 yards silver tulle Bought for wedd...",2.079442,Kids/Toys/Arts & Crafts/missing
336347,336347,Kate Spade Samsung galaxy s6 edge cases,3,"Electronics/Cell Phones & Accessories/Cases, C...",Kate Spade,23.0,1,No description yet,3.178054,"Electronics/Cell Phones & Accessories/Cases, C..."
378265,378265,Estée Lauder Bundle For Lupe,1,Beauty/Skin Care/Face,Estee Lauder,23.0,1,2 mascaras and one daywear cream.,3.178054,Beauty/Skin Care/Face/Estee Lauder
655062,655062,Pokemkon Shining Charizard Mint,1,Kids/Toys/Games,Pokemon,60.0,1,This is a mint condition shining charizard pok...,4.110874,Kids/Toys/Games/Pokemon
1473779,1473779,Listing for Carlye,2,Women/Swimwear/Two-Piece,missing,16.0,0,No description yet,2.833213,Women/Swimwear/Two-Piece/missing


In [10]:
# Make new feature: cat/subcat_1/subcat_2/brand\
train['category_brand'] = train['category_name'] + '/' + train['brand_name']
test['category_brand'] = test['category_name'] + '/' + test['brand_name']
dvalid['category_brand'] = dvalid['category_name'] + '/' + dvalid['brand_name']

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# GP

In [ ]:
#PROCESS CATEGORICAL DATA
print("Handling categorical variables...")
le = LabelEncoder()


le.fit(np.hstack([train.category_brand, test.category_brand]))
train.category_brand = le.transform(train.category_brand)
test.category_brand = le.transform(test.category_brand)
del le

train.head(3)

In [5]:
cat_brand_groups = train.groupby('category_brand')

In [50]:
#cat_brand_groups.describe()
groups_dict = {}
for name, group in cat_brand_groups:
    # Try target and price--> if price, then inference is log1p(group_dict[test_feature]['mean'])
    #groups_dict[name] = group.price.describe()
    groups_dict[name] = group.target.describe()

In [ ]:
groups_dict.keys()

In [ ]:
groups_dict['Women/Tops & Blouses/Knit Top/Arden B']['mean']

In [52]:
groups_dict

{"Women/Women's Accessories/Watches/Daniel Wellington": count    8.000000
 mean     4.064779
 std      0.449732
 min      3.401197
 25%      3.799874
 50%      4.053106
 75%      4.279693
 max      4.875197
 Name: target, dtype: float64,
 'Electronics/Cell Phones & Accessories/Cell Phone Accessories/Insignia': count    2.000000
 mean     2.238668
 std      0.225181
 min      2.079442
 25%      2.159055
 50%      2.238668
 75%      2.318282
 max      2.397895
 Name: target, dtype: float64,
 'Beauty/Makeup/Face/Boscia': count    1.000000
 mean     3.091042
 std           NaN
 min      3.091042
 25%      3.091042
 50%      3.091042
 75%      3.091042
 max      3.091042
 Name: target, dtype: float64,
 'Women/Pants/Cargo/UNIONBAY': count    5.000000
 mean     2.500132
 std      0.292353
 min      2.197225
 25%      2.197225
 50%      2.564949
 75%      2.708050
 max      2.833213
 Name: target, dtype: float64,
 'Women/Pants/Khakis, Chinos/Lee': count    1.000000
 mean     2.944439
 std     

In [47]:

def infer_GP(feature, GP):
    if(feature in GP):
        return True, GP[feature]
    else:
        return False, None

test_feature = 'Women/Skirts/Maxi/Calvin Klein'

found, result_price = infer_GP(test_feature, groups_dict)
if(found):
    print('Found')
    print(result_price['mean'])
else:
    print('Not Found, Run GRU')

Not Found, Run GRU


In [53]:
val_features = dvalid['category_brand']
#print(test_features)
y_val_pred = []
not_found = 0
count = 0
y_true = []
for val_feature in val_features:   
    
    found, result_price = infer_GP(val_feature, groups_dict)
    if(found):
        y_val_pred.append(np.exp(result_price['mean'])-1)
        #_val_pred.append(result_price['mean'])
        #y_true.append(dvalid.ix[count].price)
        #y_true.append(dvalid.iloc[[count]].price)
        y_true.append(np.array(dvalid.iloc[[count]].price)[0])
    else:
        not_found += 1
        #preds.append(-1)
        #print('Not Found, Run GRU ' + str(not_found) + ' ' + str(not_found) + ' ' + test_feature)
    count += 1
print('Not Found, Run GRU ' + str(not_found) + ' ' + str(count))

Not Found, Run GRU 0 14826


In [36]:
x = dvalid.iloc[[0]].price
np.array(x)[0]


31.0

In [19]:
import math
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5

In [54]:
v_rmsle = rmsle(y_true, y_val_pred)
print(" RMSLE error on dev test: "+str(v_rmsle))

 RMSLE error on dev test: 0.5806628738554497


In [38]:
print(len(y_val_pred))
print(y_val_pred)

14826
[22.847894155542708, 14.892154290900002, 25.031939715540222, 15.877669849254882, 28.086720643675996, 8.8514285288905725, 16.321662506886806, 18.685920875760999, 10.821831658750609, 16.77970106955107, 38.802026470764517, 13.876684080487076, 31.985249981841172, 10.40096433664081, 29.081016617316855, 22.659329188801649, 10.351480911790508, 25.316488444521077, 40.108782255053455, 14.866003355783539, 15.197450793599216, 18.88138067379111, 43.100505920988901, 29.558208683326409, 14.312616165919147, 34.314330935149371, 23.371977235710926, 15.191643424360283, 16.018021122555162, 127.5507535779295, 18.411957686353819, 63.809694736990906, 24.749428463598058, 29.081016617316855, 17.900183824241147, 29.558208683326409, 27.982753492378876, 40.108782255053455, 19.571675462506601, 15.943023835973889, 13.694540385917877, 22.764020732115537, 17.900183824241147, 21.838514724393541, 16.767924602735235, 10.976217732842029, 22.080886514576967, 13.754752027026445, 12.993917770633287, 19.88311998210272

In [39]:
print(len(y_true))
print(y_true)


[31.0, 15.0, 26.0, 16.0, 50.0, 7.0, 23.0, 23.0, 60.0, 16.0, 74.0, 12.0, 22.0, 8.0, 34.0, 16.0, 23.0, 26.0, 61.0, 256.0, 4.0, 32.0, 56.0, 40.0, 24.0, 45.0, 20.0, 12.0, 18.0, 140.0, 33.0, 46.0, 20.0, 25.0, 8.0, 36.0, 34.0, 96.0, 11.0, 10.0, 10.0, 19.0, 10.0, 14.0, 14.0, 6.0, 23.0, 14.0, 12.0, 3.0, 11.0, 93.0, 3.0, 12.0, 14.0, 12.0, 24.0, 100.0, 14.0, 12.0, 9.0, 16.0, 35.0, 11.0, 44.0, 10.0, 26.0, 9.0, 9.0, 35.0, 55.0, 16.0, 32.0, 7.0, 10.0, 100.0, 19.0, 11.0, 34.0, 9.0, 12.0, 26.0, 4.0, 39.0, 12.0, 22.0, 10.0, 9.0, 55.0, 31.0, 8.0, 6.0, 14.0, 8.0, 11.0, 8.0, 20.0, 11.0, 14.0, 10.0, 35.0, 206.0, 5.0, 42.0, 15.0, 50.0, 8.0, 36.0, 9.0, 10.0, 10.0, 16.0, 16.0, 39.0, 15.0, 7.0, 16.0, 3.0, 12.0, 51.0, 25.0, 12.0, 15.0, 46.0, 10.0, 7.0, 10.0, 10.0, 16.0, 10.0, 14.0, 14.0, 36.0, 19.0, 3.0, 16.0, 9.0, 42.0, 15.0, 31.0, 10.0, 19.0, 14.0, 307.0, 11.0, 6.0, 5.0, 17.0, 8.0, 10.0, 10.0, 21.0, 15.0, 50.0, 17.0, 37.0, 24.0, 60.0, 10.0, 129.0, 10.0, 7.0, 34.0, 10.0, 53.0, 34.0, 11.0, 14.0, 14.0, 16.0, 33

In [43]:
# Get predictions on test
test_features = test['category_brand']
#print(test_features)
y_pred = []
not_found = 0
count = 0
for test_feature in test_features:   
    
    found, result_price = infer_GP(test_feature, groups_dict)
    if(found):
        y_pred.append(np.exp(result_price['mean'])-1)
        #y_pred.append(result_price['mean'])

    else:
        not_found += 1
        y_pred.append(-1)
        #print('Not Found, Run GRU ' + str(not_found) + ' ' + str(not_found) + ' ' + test_feature)
        
    count += 1
print('Not Found, Run GRU ' + str(not_found) + ' ' + str(count))

submission_GP = test[["test_id"]]
submission_GP["price"] = y_pred

Not Found, Run GRU 9509 693359


C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


# GRU

In [ ]:
#PROCESS CATEGORICAL DATA
print("Handling categorical variables...")
le = LabelEncoder()

le.fit(np.hstack([train.category_name, test.category_name]))
train.category_name = le.transform(train.category_name)
test.category_name = le.transform(test.category_name)

le.fit(np.hstack([train.brand_name, test.brand_name]))
train.brand_name = le.transform(train.brand_name)
test.brand_name = le.transform(test.brand_name)
del le


le.fit(np.hstack([train.category_brand, test.category_brand]))
train.category_brand = le.transform(train.category_brand)
test.category_brand = le.transform(test.category_brand)
del le

train.head(3)

In [ ]:
#PROCESS TEXT: RAW
print("Text to seq process...")
from keras.preprocessing.text import Tokenizer
raw_text = np.hstack([train.item_description.str.lower(), train.name.str.lower()])

print("   Fitting tokenizer...")
tok_raw = Tokenizer()
tok_raw.fit_on_texts(raw_text)
print("   Transforming text to seq...")

train["seq_item_description"] = tok_raw.texts_to_sequences(train.item_description.str.lower())
test["seq_item_description"] = tok_raw.texts_to_sequences(test.item_description.str.lower())
train["seq_name"] = tok_raw.texts_to_sequences(train.name.str.lower())
test["seq_name"] = tok_raw.texts_to_sequences(test.name.str.lower())
train.head(3)

In [ ]:
#SEQUENCES VARIABLES ANALYSIS
max_name_seq = np.max([np.max(train.seq_name.apply(lambda x: len(x))), np.max(test.seq_name.apply(lambda x: len(x)))])
max_seq_item_description = np.max([np.max(train.seq_item_description.apply(lambda x: len(x)))
                                   , np.max(test.seq_item_description.apply(lambda x: len(x)))])
print("max name seq "+str(max_name_seq))
print("max item desc seq "+str(max_seq_item_description))

In [ ]:
#EMBEDDINGS MAX VALUE
#Base on the histograms, we select the next lengths
MAX_NAME_SEQ = 10
MAX_ITEM_DESC_SEQ = 75
MAX_TEXT = np.max([np.max(train.seq_name.max())
                   , np.max(test.seq_name.max())
                  , np.max(train.seq_item_description.max())
                  , np.max(test.seq_item_description.max())])+2
MAX_CATEGORY = np.max([train.category_name.max(), test.category_name.max()])+1
MAX_BRAND = np.max([train.brand_name.max(), test.brand_name.max()])+1
MAX_CONDITION = np.max([train.item_condition_id.max(), test.item_condition_id.max()])+1

In [ ]:
#EXTRACT DEVELOPTMENT TEST
dtrain, dvalid = train_test_split(train, random_state=123, train_size=0.99)
print(dtrain.shape)
print(dvalid.shape)

In [ ]:
#KERAS DATA DEFINITION
from keras.preprocessing.sequence import pad_sequences

def get_keras_data(dataset):
    X = {
        'name': pad_sequences(dataset.seq_name, maxlen=MAX_NAME_SEQ)
        ,'item_desc': pad_sequences(dataset.seq_item_description, maxlen=MAX_ITEM_DESC_SEQ)
        ,'brand_name': np.array(dataset.brand_name)
        ,'category_name': np.array(dataset.category_name)
        ,'item_condition': np.array(dataset.item_condition_id)
        ,'num_vars': np.array(dataset[["shipping"]])
    }
    return X

X_train = get_keras_data(dtrain)
X_valid = get_keras_data(dvalid)
X_test = get_keras_data(test)

In [ ]:
#KERAS MODEL DEFINITION
from keras.layers import Input, Dropout, Dense, BatchNormalization, Activation, concatenate, GRU, Embedding, Flatten, BatchNormalization
from keras.models import Model
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from keras import backend as K

def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]

def rmsle_cust(y_true, y_pred):
    first_log = K.log(K.clip(y_pred, K.epsilon(), None) + 1.)
    second_log = K.log(K.clip(y_true, K.epsilon(), None) + 1.)
    return K.sqrt(K.mean(K.square(first_log - second_log), axis=-1))

def get_model():
    #params
    dr_r = 0.1
    
    #Inputs
    name = Input(shape=[X_train["name"].shape[1]], name="name")
    item_desc = Input(shape=[X_train["item_desc"].shape[1]], name="item_desc")
    brand_name = Input(shape=[1], name="brand_name")
    category_name = Input(shape=[1], name="category_name")
    item_condition = Input(shape=[1], name="item_condition")
    num_vars = Input(shape=[X_train["num_vars"].shape[1]], name="num_vars")
    
    #Embeddings layers
    emb_name = Embedding(MAX_TEXT, 50)(name)
    emb_item_desc = Embedding(MAX_TEXT, 50)(item_desc)
    emb_brand_name = Embedding(MAX_BRAND, 10)(brand_name)
    emb_category_name = Embedding(MAX_CATEGORY, 10)(category_name)
    emb_item_condition = Embedding(MAX_CONDITION, 5)(item_condition)
    
    #rnn layer
    rnn_layer1 = GRU(16) (emb_item_desc)
    rnn_layer2 = GRU(8) (emb_name)
    
    #main layer
    main_l = concatenate([
        Flatten() (emb_brand_name)
        , Flatten() (emb_category_name)
        , Flatten() (emb_item_condition)
        , rnn_layer1
        , rnn_layer2
        , num_vars
    ])
    main_l = Dropout(dr_r) (Dense(128) (main_l))
    main_l = Dropout(dr_r) (Dense(64) (main_l))
    
    #output
    output = Dense(1, activation="linear") (main_l)
    
    #model
    model = Model([name, item_desc, brand_name
                   , category_name, item_condition, num_vars], output)
    model.compile(loss="mse", optimizer="adam", metrics=["mae", rmsle_cust])
    
    return model

    
model = get_model()
model.summary()
'''
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
'''

In [ ]:
#FITTING THE MODEL
BATCH_SIZE = 20000
epochs = 5

model = get_model()
model.fit(X_train, dtrain.target, epochs=epochs, batch_size=BATCH_SIZE
          , validation_data=(X_valid, dvalid.target)
          , verbose=1)

In [ ]:
def rmsle(y, y_pred):
    assert len(y) == len(y_pred)
    to_sum = [(math.log(y_pred[i] + 1) - math.log(y[i] + 1)) ** 2.0 for i,pred in enumerate(y_pred)]
    return (sum(to_sum) * (1.0/len(y))) ** 0.5
#Source: https://www.kaggle.com/marknagelberg/rmsle-function

#EVLUEATE THE MODEL ON DEV TEST: What is it doing?
val_preds = model.predict(X_valid)
val_preds = target_scaler.inverse_transform(val_preds)
val_preds = np.exp(val_preds)+1

#mean_absolute_error, mean_squared_log_error
y_true = np.array(dvalid.price.values)
y_pred = val_preds[:,0]
v_rmsle = rmsle(y_true, y_pred)
print(" RMSLE error on dev test: "+str(v_rmsle))

# Submission

In [ ]:
#CREATE PREDICTIONS
preds = model.predict(X_test, batch_size=BATCH_SIZE)
preds = target_scaler.inverse_transform(preds)
preds = np.exp(preds)-1

submission = test[["test_id"]]
submission["price"] = preds

In [ ]:
submission.to_csv("./submission_GRU_GP.csv", index=False)
submission.price.hist()